# How to save parameters along with the function 

This is a workaround for the problem of how to save *and restore* the parameters of a workflow (e.g., sigma for blurring, threshold values, etc) along with the functions. The idea is simple:

Instead of setting workflow steps with, for instance, `w.set('blurring', blur_function, 'input', sigma=5)`, this notebook creates a partial function of `blur_function()` which is then added to the workflow. When the workflow is loaded and the partial function is passed to magicgui, the parameters are restored correctly.

In [ ]:
from napari_workflows import Workflow
from napari.types import ImageData
import napari
import yaml
from skimage.filters import threshold_otsu, gaussian

import inspect
import magicgui
from functools import partial
from napari_tools_menu import make_gui

### First fix:

Turn function into partial object before putting it into the workflow

In [ ]:
w = Workflow()

# define segmentation
def threshold(image: ImageData, value: float = 1) -> ImageData:
    return image > value

def add(imageA: ImageData, imageB:ImageData) -> ImageData:
    return imageA + imageB

def blur(image: ImageData, sigma: float=2):
    return gaussian(image, sigma=sigma)

kwargs = {'sigma': 5}

_blur = partial(blur, **kwargs)

w.set('blurring', _blur, 'input', **kwargs)
w.set('binarization', threshold, 'blurring')


with open('workflow.yaml', 'w') as f:
    yaml.dump(w, f)

In [ ]:
with open("workflow.yaml", "rb") as stream:
    w=yaml.unsafe_load(stream)
    
w._tasks

In [ ]:
viewer = napari.Viewer()
steps = w._tasks.keys()

for key in w._tasks.keys():
    func = w._tasks[key][0]
    widget = magicgui.magicgui(func)
    viewer.window.add_dock_widget(widget)

### Second fix:
Turn function into partial object after loading it

In [ ]:
w = Workflow()
w.set('blurring', blur, 'input', **kwargs)
w.set('binarization', threshold, 'blurring')

with open('_workflow.yaml', 'w') as f:
    yaml.dump(w, f)

In [ ]:
with open("_workflow.yaml", "rb") as stream:
    w=yaml.unsafe_load(stream)
    
w._tasks

In [ ]:
viewer = napari.Viewer()

for task in w._tasks.keys():
    func = w._tasks[task][0]  # extract function from workflow
    keyword_list = list(inspect.signature(func).parameters.keys())  # get name of kwargs
    
    # We need to retrieve a dictionary with parameter: value from the saved function
    kw_dict = {}
    for kw, val in zip(keyword_list, w._tasks[task][1:]):
        kw_dict[kw] = val
    kw_dict.pop('image')
    
    func = partial(w._tasks[task][0], **kw_dict)
    
    widget = make_gui(func, viewer)
    viewer.window.add_dock_widget(widget)